自这本书开始以来，你已经走了很远。您现在可以训练图像分类模型、图像分割模型、矢量数据分类或回归模型、时间序列预测模型、文本分类模型、序列到序列模型，甚至文本和图像的生成模型。你已经覆盖了所有基础。

然而，到目前为止，您的模型都经过了小规模的训练——在小型数据集上，使用单个GPU——它们通常没有在我们查看的每个数据集上达到最佳的性能。毕竟，这本书是一本保守党的入门书。如果你想在现实世界中在全新的问题上取得最先进的结果，你仍然需要跨越一点鸿沟。

这倒数第二章是关于弥合这一差距，并为您提供从机器学习学生到成熟的机器学习工程师所需的最佳实践。 我们将回顾系统改进模型性能的基本技术：hyperparameter tuning 和 model ensembling。然后，我们将研究如何通过多GPU和TPU训练、混合精度和利用云端远程计算资源来加快和扩展模型训练。

# 13.1 Getting the most out of your models

如果您只是需要正常工作的东西，尝试不同的架构配置就足够了。在本节中，我们将通过一套构建最先进的深度学习模型的必备技术，从 “works okay” 到 “works great and wins machine learning competitions”。

## 13.1.1 Hyperparameter tuning

在构建深度学习模型时，您必须做出许多看似武断的决定：您应该堆叠多少层？每层应该放多少个单元或 filter？ 您应该使用relu作为激活，还是使用其他函数？您应该在给定层后使用 `BatchNormalization` 吗？你应该用多少 `dropout`？等等。这些架构级参数被称为超参数，以区别于模型的参数，模型的参数是通过反向传播训练的。

在实践中，经验丰富的机器学习工程师和研究人员随着时间的推移构建直觉，了解在这些选择时哪些有效，哪些无效——他们开发了 hyperparameter-tuning 技能。但是没有正式的规则。如果您想达到给定任务所能实现的极限，您不能满足于这种任意的选择。即使你有非常好的直觉，你的初始决定几乎总是次优。您可以通过手动调整它们并反复重新训练模型来完善您的选择——这是机器学习工程师和研究人员花费的大部分时间所做的。但作为人类，整天摆弄超参数不应该是你的工作——最好留给机器。

因此，您需要以原则性的方式自动探索可能决策的空间。您需要搜索架构空间，并从实验中找到性能最好的架构。这就是自动超参数优化领域的内容：这是一个完整的研究领域，也是一个重要的研究领域。

优化超参数的过程通常如下所示：

- 选择一组超参数（自动）。
- 构建相应的模型。
- 将其与您的训练数据拟合，并衡量验证数据的性能。
- 选择下一组超参数进行尝试（自动）。
- 重复。
- 最终，测量测试数据的性能。

这个过程的关键是分析验证集性能与各种超参数值之间关系的算法，以选择下一组超参数。 许多不同的技术是可能的：贝叶斯优化、遗传算法、简单的随机搜索等。

训练模型的权重相对容易：您在小批量数据上计算损失函数，然后使用反向传播将权重向正确的方向移动。另一方面，更新超参数带来了独特的挑战。考虑以下几点：

- 超参数空间通常由离散决策组成，因此不是连续的或可微的。因此，您通常无法在超参数空间中进行梯度下降。相反，您必须依赖无梯度优化技术，这些技术自然比梯度下降效率低得多。
- 计算此优化过程的反馈信号（这组超参数是否会导致此任务的高性能模型？）可能非常昂贵：它需要在数据集上从头开始创建和训练新模型。
- 反馈信号可能很嘈杂：如果训练运行表现更好0.2%，是因为更好的模型配置，还是因为你幸运地使用初始化权重值？

谢天谢地，有一个工具可以简化超参数调优：KerasTuner。我们来看看吧。

### USING KERASTUNER

KerasTuner允许您用一系列可能的选择替换硬编码的超参数值，例如 `units=32`，例如 `Int(name="units", min_value=16, max_value=64, step=16)`。给定模型中的这组选择称为超参数调优过程的搜索空间。

要指定搜索空间，请定义模型构建函数（请参阅下一个列表）。它需要一个 `hp` 参数，您可以从中采样超参数范围，并返回编译的 Keras 模型。

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

如果您想采用更模块化和可配置的模型构建方法，您还可以对 `HyperModel` 类进行子类，并定义构建方法，如下所示。

In [5]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes): 
        self.num_classes = num_classes
        
    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16) 
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

/var/folders/7q/gtdgjdyn4rn030xffh9zbbrm0000gp/T/ipykernel_18632/4065724612.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


下一步是定义一个“tuner”。从原理上讲，你可以把 tuner 想象成一个循环，它会反复:

- 选择一组超参数值
- 使用这些值调用模型构建函数来创建模型
- 训练模型并记录其指标

KerasTuner有几个内置 tuner —— `RandomSearch`、`BayesianOptimization` 和 `Hyperband`。让我们试试贝叶斯优化，这是一个tuner，它试图做出智能预测，相比于之前选择的结果，新的超参数值可能表现最好：

In [6]:
tuner = kt.BayesianOptimization(
     build_model,
     objective="val_accuracy",
     max_trials=100,
     executions_per_trial=2,
     directory="mnist_kt_test",
     overwrite=True,
)

Metal device set to: Apple M1


2022-04-27 19:54:58.034984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-27 19:54:58.035974: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


您可以通过 `search_space_summary()` 显示搜索空间的概述：

In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


> Objective maximization and minimization
> 对于内置指标（例如准确性，在我们的案例中），指标的方向（准确性应最大化，但损失应最小化）由KerasTuner推断。但是，对于自定义指标，您应该自己指定它，如下所示：
> ```
> objective = kt.Objective(
    name="val_accuracy",
    direction="max")
tuner = kt.BayesianOptimization(
    build_model,
    objective=objective,
    ...
)
> ```

最后，让我们开始搜索。不要忘记通过验证数据，并确保不要将测试集用作验证数据——否则您很快就会开始过拟合测试数据，您将无法再信任您的测试指标：

In [8]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:] 
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:] 
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 100 Complete [00h 01m 03s]
val_accuracy: 0.9731000661849976

Best val_accuracy So Far: 0.9771000444889069
Total elapsed time: 02h 01m 51s
INFO:tensorflow:Oracle triggered exit


前面的示例将在几分钟内运行，因为我们只考虑了一些可能的选择，并且我们正在对MNIST进行训练。然而，对于典型的搜索空间和数据集，您通常会发现自己让超参数搜索在一夜之间甚至几天内运行。如果您的搜索过程崩溃，您可以随时重新启动它——只需在 tuner 中指定 `overwrite=False`，这样它就可以从存储在磁盘上的实验日志中恢复。

搜索完成后，您可以查询最佳超参数配置，可用于创建高性能模型，然后重新训练。

In [9]:
top_n = 4 
best_hps = tuner.get_best_hyperparameters(top_n)

通常，在重新训练这些模型时，您可能希望将验证数据作为训练数据的一部分，因为您不会进行任何进一步的超参数更改，因此您将不再评估验证数据的性能。在我们的示例中，我们将根据原始MNIST训练数据的全部来训练这些最终模型，而无需保留验证集。

然而，在我们根据完整的训练数据进行训练之前，我们需要确定最后一个参数：要训练的最佳 epoch 数。通常，您希望比搜索期间更长时间地训练新型号：在 `EarlyStopping` 回调中使用 `patience` 以节省搜索时间，但这可能会导致模型欠拟合。只需使用验证集即可找到最佳 epoch：

In [10]:
def get_best_epoch(hp): 
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1 
    print(f"Best epoch: {best_epoch}")
    return best_epoch

最后，在完整的数据集上训练的时间比这个 epoch 数长一点，因为您正在训练更多的数据；在这个例子中，要多20%：

In [11]:
def get_best_trained_model(hp): 
    best_epoch = get_best_epoch(hp) 
    model.fit(
        x_train_full, y_train_full,
    batch_size=128, epochs=int(best_epoch * 1.2)) 
    return model

best_models = [] 
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)


Epoch 1/100


2022-04-27 23:56:54.701010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


387/391 [============================>.] - ETA: 0s - loss: 0.4230 - accuracy: 0.8854

2022-04-27 23:56:56.813360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 3s 5ms/step - loss: 0.4211 - accuracy: 0.8859 - val_loss: 0.2371 - val_accuracy: 0.9345
Epoch 2/100
391/391 [==============================] - 2s 5ms/step - loss: 0.2173 - accuracy: 0.9381 - val_loss: 0.1796 - val_accuracy: 0.9506
Epoch 3/100
391/391 [==============================] - 2s 4ms/step - loss: 0.1667 - accuracy: 0.9523 - val_loss: 0.1530 - val_accuracy: 0.9560
Epoch 4/100
391/391 [==============================] - 2s 5ms/step - loss: 0.1374 - accuracy: 0.9602 - val_loss: 0.1333 - val_accuracy: 0.9625
Epoch 5/100
391/391 [==============================] - 2s 5ms/step - loss: 0.1172 - accuracy: 0.9655 - val_loss: 0.1237 - val_accuracy: 0.9633
Epoch 6/100
391/391 [==============================] - 2s 5ms/step - loss: 0.1023 - accuracy: 0.9700 - val_loss: 0.1131 - val_accuracy: 0.9652
Epoch 7/100
391/391 [==============================] - 2s 5ms/step - loss: 0.0909 - accuracy: 0.9739 - val_loss: 0.1052 - val_accuracy: 0.9697
Epoch 8/100

NameError: name 'model' is not defined

请注意，如果您不担心性能略有不足，您可以采取一条捷径：只需使用 tuner 重新加载在超参数搜索中保存的表现最好模型的权重， 而不需要从头训练新的模型。

In [ ]:
best_models = tuner.get_best_models(top_n)

**Note：** 大规模进行自动超参数优化时需要考虑的一个重要问题是验证集的过拟合。 由于您正在根据使用验证数据计算的信号更新超参数，因此您正在对验证数据进行有效训练，因此它们将很快过拟合验证数据。时刻牢记这一点。

### THE ART OF CRAFTING THE RIGHT SEARCH SPACE

总体而言，超参数优化是一种强大的技术，是在任何任务中获得最先进的模型或赢得机器学习比赛的绝对要求。想想看：从前，人们手工制作了浅层机器学习模型的特征。这非常不理想。现在，深度学习使分层特征工程的任务自动化——这些特征是使用反馈信号学习的，而不是手工调整的，这是应该的。同样，您不应该手工制作模型架构；您应该以原则性的方式优化它们。

然而，进行超参数调优并不能取代熟悉模型架构的最佳实践。搜索空间随着选择数量的增多而增大， 因此，将所有内容转换为超参数并让 tuner 进行整理就太贵了。您需要明智地设计正确的搜索空间。超参数调优是自动化，而不是魔法：您用它来自动化本来可以手动运行的实验，但您仍然需要精心挑选有可能产生良好指标的实验配置。

好消息是，通过利用超参数调优，您必须从微观决策中做出配置决策（我为这一层选择多少单元？）到更高级别的架构决策（我应该在整个模型中使用残差连接吗？）。虽然微观决策是特定于特定模型和特定数据集的，但更高级别的决策在不同的任务和数据集中更普遍。例如，几乎每个图像分类问题都可以通过相同的搜索空间模板来解决。

根据这一逻辑，KerasTuner试图提供与图像分类等广泛问题相关的预制搜索空间。只需添加数据，运行搜索，并获得一个相当不错的模型。您可以尝试 hypermodels `kt.applications.HyperXception` 和`kt.applications.HyperResNet`，它们是 Keras Applications 模型的有效 tunable 版本。

**THE FUTURE OF HYPERPARAMETER TUNING: AUTOMATED MACHINE LEARNING**

目前，您作为深度学习工程师的大部分工作包括使用Python脚本训练数据，然后长时间调整深度网络工作的架构和超参数，以获得有效的模型，甚至获得最先进的模型，如果您雄心勃勃的话。不用说，这不是一个最佳设置。但自动化可以有所帮助，它不会仅仅停止在超参数调优。

搜索一组可能的学习速率或可能的层大小只是第一步。我们也可以更加雄心勃勃，并试图从头开始生成模型架构本身，尽可能少地限制，例如通过强化学习或遗传算法。将来，整个端到端机器学习 pipeline 将自动生成，而不是由工程师手工制作。这被称为自动机器学习或AutoML。您已经可以利用AutoKeras（[https://github.com/keras-team/autokeras](https://github.com/keras-team/autokeras)） 等库来解决基本的机器学习问题，而您很少参与其中。 

今天，AutoML 仍处于早期阶段，它不会扩展到大问题。但是，当 AutoML 足够成熟，可以广泛采用时，机器学习工程师的工作不会消失 —— 反之，工程师将向价值创造链向上移动。他们将开始在数据管理上投入更多精力，制作真正反映业务目标的复杂损失函数，并了解他们的模型如何影响部署他们的数字生态系统（例如消耗模型预测并生成模型训练数据的用户）。这些问题目前只有最大的公司才能考虑。

始终着眼于大局，专注于了解基础知识，并牢记高度专业化的单调乏味最终会自动化消失。将其看做礼物——提高工作流程的生产力，而不是对自己的权利构成威斯。无休止地调整旋钮不应该是你的工作。

## 13.1.2 Model ensembling

在任务上获得最佳结果的另一种强大技术是 `model ensembling`。 集成包括将一组不同模型的预测汇集在一起，以产生更好的预测。如果您查看机器学习竞赛，特别是在Kaggle上，您会发现获胜者使用非常大的模型组合，无论多么优秀，这些模型都不可避免地击败任何单个模型。

组装依赖于以下假设，即独立训练的不同表现良好的模型可能出于不同的原因：每个模型都查看数据的略有不同的方面来进行预测，获得部分“真相”，但不是全部。你可能熟悉盲人和大象的古老寓言：一群盲人第一次遇到大象，并试图通过触摸大象来了解大象是什么。每个人都触摸大象身体的不同部位——只有一个部分，如躯干或腿。然后，男人们互相描述大象是什么：“它就像一条蛇”、“像一根柱子或一棵树”，等等。盲人本质上是机器学习模型，试图从自己的角度理解训练数据，每个模型都使用自己的假设（由模型的独特架构和独特的初始化权重提供）。他们每个人都得到了数据的部分真相，但不是全部真相。通过将他们的观点汇集在一起，您可以对数据进行更准确的描述。大象是部分的组合：没有一个盲人完全正确，但经过一起采访，他们可以讲述一个相当准确的故事。

让我们以分类为例。汇集一组分类器（集成分类器）预测的最简单方法是在推断时间平均他们的预测：

```python
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)
final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)
```

然而，只有当分类器或多或少同样好时，这才会起作用。如果其中一个明显比其他的差，最终预测可能不如小组中最好的分类器好。集成分类器的一个更明智的方法是做一个加权平均值，其中权重是在验证数据上学习的——通常，越好的分类器被赋予更高的权重，而较差的分类器的权重较低。要搜索一组好的集成权重，您可以使用随机搜索或简单的优化算法，例如Nelder-Mead算法：

```python
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)
final_preds = 0.5 * preds_a + 0.25 * preds_b + 0.1 * preds_c + 0.15 * preds_d
```

有许多可能的变体：例如，您可以对预测进行指数的平均。一般来说，在验证数据上简单加权平均提供了一个非常强大的基线。使集成工作的关键是分类器集的多样性。多样性就是力量。如果所有盲人只摸大象的尾巴，他们就会同意大象就像蛇，他们将永远对大象的真相一无所知。多样性是集成奏效的原因。在机器学习方面，如果您的所有模型都以相同的方式有偏差，您的集成将保留相同的偏差。如果您的模型以不同的方式有偏差，偏差将相互抵消，集成将更加坚固和准确。

出于这个原因，您应该集成尽可能好的模型，同时尽可能不同。这通常意味着使用非常不同的架构，甚至不同种类的机器学习方法。一件基本上不值得做的事情是，从不同的随机初始化中集成了多次独立训练的同一网络。如果您的模型之间唯一的区别是它们的运行初始化和它们暴露于训练数据的顺序，那么您的集成将是低多样性的，与任何单个模型相比，只会提供微小的改进。

我发现在实践中效果良好的一件事——但这并没有推广到每个问题领域——是使用一系列基于树的方法（如随机森林或梯度增强树）和深层神经网络。2014年，Andrey Kolev和我使用各种树模型和深层神经网络在Kaggle（[www.kaggle.com/c/higgs-boson](www.kaggle.com/c/higgs-boson)） 的Higgs Boson衰变检测挑战赛中排名第四。值得注意的是，集成中的一个模型起源于与其他模型不同的方法（这是一个正则化的贪婪森林），它的得分比其他模型差得多。毫不奇怪，它在集成中被赋予了很小的权重。但令我们惊讶的是，事实证明，它在很大程度上改进了整体集成，因为它与所有其他模型都非常不同：它提供了其他模型无法访问的信息。这正是集成闪亮的意义。这与其说是关于你最好的模型有多好，不如说是关于你一组候选模型的多样性。